In [9]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy import stats

In [2]:
# specify file paths
train_df = "../.data/train.csv"
train_targets = "../.data/train_targets.csv"

# read in files
X = pd.read_csv(train_df)
X = X.iloc[:, 1:]                       # remove cell line labels
y = pd.read_csv(train_targets)['AAC']   # keep only AAC column

# preview
print("Train Features Shape:", X.shape)
print("Train Targets Shape:", y.shape)

Train Features Shape: (742, 19920)
Train Targets Shape: (742,)


In [3]:
# Split the training dataframe into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# DMatrix for XGBoost
train_dmatrix = xgb.DMatrix(X_train, label=y_train)
val_dmatrix = xgb.DMatrix(X_test, label=y_test)

In [5]:
# XGBoost params
params = {
    'objective': 'reg:squarederror',  # regression
    'eval_metric': 'rmse',
    'max_depth': 5,
    'eta': 0.1,
    'subsample': 0.8
}

In [6]:
# train
watchlist = [(train_dmatrix, 'train'), (val_dmatrix, 'eval')]
model = xgb.train(params, train_dmatrix, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=True)

[0]	train-rmse:0.09852	eval-rmse:0.08631
[1]	train-rmse:0.09283	eval-rmse:0.08441
[2]	train-rmse:0.08773	eval-rmse:0.08387
[3]	train-rmse:0.08356	eval-rmse:0.08217
[4]	train-rmse:0.07966	eval-rmse:0.08111
[5]	train-rmse:0.07547	eval-rmse:0.08016
[6]	train-rmse:0.07180	eval-rmse:0.07935
[7]	train-rmse:0.06829	eval-rmse:0.07824
[8]	train-rmse:0.06473	eval-rmse:0.07858
[9]	train-rmse:0.06150	eval-rmse:0.07801
[10]	train-rmse:0.05922	eval-rmse:0.07827
[11]	train-rmse:0.05663	eval-rmse:0.07832
[12]	train-rmse:0.05470	eval-rmse:0.07831
[13]	train-rmse:0.05242	eval-rmse:0.07850
[14]	train-rmse:0.05023	eval-rmse:0.07843
[15]	train-rmse:0.04858	eval-rmse:0.07815
[16]	train-rmse:0.04692	eval-rmse:0.07820
[17]	train-rmse:0.04495	eval-rmse:0.07798
[18]	train-rmse:0.04284	eval-rmse:0.07805
[19]	train-rmse:0.04135	eval-rmse:0.07779
[20]	train-rmse:0.04014	eval-rmse:0.07773
[21]	train-rmse:0.03896	eval-rmse:0.07786
[22]	train-rmse:0.03773	eval-rmse:0.07775
[23]	train-rmse:0.03621	eval-rmse:0.07753
[2

In [7]:
# make preds on test set
y_pred = model.predict(val_dmatrix)

In [10]:
# eval RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Test RMSE:", rmse)

Test RMSE: 0.07739229844278923


/home/vivian.chu/conda/envs/xai/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
# spearman corr
res = stats.spearmanr(y_pred, y_test)
print("Spearman's correlation:", res[0])

Spearman's correlation: 0.42278471139570384
